In [1]:
import numpy as np
from pynq import allocate
from pynq import Overlay
import time

In [26]:
def generate_random_qubit():
    # Generate random real and imaginary parts for alpha and beta
    a_real = np.random.uniform(-1, 1)
    a_imag = np.random.uniform(-1, 1)
    b_real = np.random.uniform(-1, 1)
    b_imag = np.random.uniform(-1, 1)
    
    # Create complex numbers for alpha and beta
    alpha = complex(a_real, a_imag)
    beta = complex(b_real, b_imag)
    
    # Normalise to ensure |alpha|^2 + |beta|^2 = 1
    norm = np.sqrt(abs(alpha)**2 + abs(beta)**2)
    alpha /= norm
    beta /= norm
    
    # Output real and imaginary parts separately
    return alpha.real, alpha.imag, beta.real, beta.imag

# Generate and print a random qubit
alpha_real, alpha_imag, beta_real, beta_imag = generate_random_qubit()
print("Qubit 1: Alpha (Real):", alpha_real)
print("Qubit 1: Alpha (Imaginary):", alpha_imag)
print("Qubit 1: Beta (Real):", beta_real)
print("Qubit 1: Beta (Imaginary):", beta_imag)

# Generate and print a second random qubit
alpha_real2, alpha_imag2, beta_real2, beta_imag2 = generate_random_qubit()
print("Qubit 2: Alpha (Real):", alpha_real2)
print("Qubit 2: Alpha (Imaginary):", alpha_imag2)
print("Qubit 2: Beta (Real):", beta_real2)
print("Qubit 2: Beta (Imaginary):", beta_imag2)

def generate_one_qubit():
    # Set alpha to 0 and beta to 1 for the |1> state
    alpha_real = 0.0
    alpha_imag = 0.0
    beta_real = 1.0
    beta_imag = 0.0
    
    # Return the components of the qubit
    return alpha_real, alpha_imag, beta_real, beta_imag

# Generate and print a qubit in the |1> state
alpha_real1, alpha_imag1, beta_real1, beta_imag1 = generate_one_qubit()
print("Control Qubit: Alpha (Real):", alpha_real1)
print("Control Qubit: Alpha (Imaginary):", alpha_imag1)
print("Control Qubit: Beta (Real):", beta_real1)
print("Control Qubit: Beta (Imaginary):", beta_imag1)



Qubit 1: Alpha (Real): -0.6351942356700025
Qubit 1: Alpha (Imaginary): -0.4850750796694228
Qubit 1: Beta (Real): 0.21101838072796023
Qubit 1: Beta (Imaginary): -0.5627625547691087
Qubit 2: Alpha (Real): -0.6831882282075797
Qubit 2: Alpha (Imaginary): 0.12345720750125405
Qubit 2: Beta (Real): 0.20646033537979577
Qubit 2: Beta (Imaginary): 0.6894826268075523
Control Qubit: Alpha (Real): 0.0
Control Qubit: Alpha (Imaginary): 0.0
Control Qubit: Beta (Real): 1.0
Control Qubit: Beta (Imaginary): 0.0


In [102]:
def convert_to_unsigned_fixed_point(alpha_real, alpha_imag, beta_real, beta_imag):
    # Convert each component to 32-bit signed fixed-point representation
    fixed_alpha_real = np.int32(alpha_real * (2**31))
    fixed_alpha_imag = np.int32(alpha_imag * (2**31))
    fixed_beta_real = np.int32(beta_real * (2**31))
    fixed_beta_imag = np.int32(beta_imag * (2**31))
    
    # Convert to unsigned by adding 2^32 to negative values
    unsigned_alpha_real = np.uint32(fixed_alpha_real if fixed_alpha_real >= 0 else fixed_alpha_real + 2**32)
    unsigned_alpha_imag = np.uint32(fixed_alpha_imag if fixed_alpha_imag >= 0 else fixed_alpha_imag + 2**32)
    unsigned_beta_real = np.uint32(fixed_beta_real if fixed_beta_real >= 0 else fixed_beta_real + 2**32)
    unsigned_beta_imag = np.uint32(fixed_beta_imag if fixed_beta_imag >= 0 else fixed_beta_imag + 2**32)
    
    # Pack into a 1D 128-bit array (4 x 32-bit unsigned integers)
    unsigned_fixed_point_array = np.array([unsigned_alpha_real, unsigned_alpha_imag, unsigned_beta_real, unsigned_beta_imag], dtype=np.uint32)
    return unsigned_fixed_point_array

# Convert the random qubit values
unsigned_fixed_point_data = convert_to_unsigned_fixed_point(alpha_real, alpha_imag, beta_real, beta_imag)
print(unsigned_fixed_point_data)

[4251253647   71844080 4220293749 4222325837]


In [103]:
def convert_to_unsigned_fixed_point_16bit(q1_alpha_real, q1_alpha_imag, q1_beta_real, q1_beta_imag,
                                           q2_alpha_real, q2_alpha_imag, q2_beta_real, q2_beta_imag):
    # Convert each component to 16-bit signed fixed-point representation
    fixed_q1_alpha_real = np.int16(q1_alpha_real * (2**15))
    fixed_q1_alpha_imag = np.int16(q1_alpha_imag * (2**15))
    fixed_q1_beta_real = np.int16(q1_beta_real * (2**15))
    fixed_q1_beta_imag = np.int16(q1_beta_imag * (2**15))

    fixed_q2_alpha_real = np.int16(q2_alpha_real * (2**15))
    fixed_q2_alpha_imag = np.int16(q2_alpha_imag * (2**15))
    fixed_q2_beta_real = np.int16(q2_beta_real * (2**15))
    fixed_q2_beta_imag = np.int16(q2_beta_imag * (2**15))

    # Convert to unsigned by adding 2^16 to negative values
    unsigned_q1_alpha_real = np.uint16(fixed_q1_alpha_real if fixed_q1_alpha_real >= 0 else fixed_q1_alpha_real + 2**16)
    unsigned_q1_alpha_imag = np.uint16(fixed_q1_alpha_imag if fixed_q1_alpha_imag >= 0 else fixed_q1_alpha_imag + 2**16)
    unsigned_q1_beta_real = np.uint16(fixed_q1_beta_real if fixed_q1_beta_real >= 0 else fixed_q1_beta_real + 2**16)
    unsigned_q1_beta_imag = np.uint16(fixed_q1_beta_imag if fixed_q1_beta_imag >= 0 else fixed_q1_beta_imag + 2**16)

    unsigned_q2_alpha_real = np.uint16(fixed_q2_alpha_real if fixed_q2_alpha_real >= 0 else fixed_q2_alpha_real + 2**16)
    unsigned_q2_alpha_imag = np.uint16(fixed_q2_alpha_imag if fixed_q2_alpha_imag >= 0 else fixed_q2_alpha_imag + 2**16)
    unsigned_q2_beta_real = np.uint16(fixed_q2_beta_real if fixed_q2_beta_real >= 0 else fixed_q2_beta_real + 2**16)
    unsigned_q2_beta_imag = np.uint16(fixed_q2_beta_imag if fixed_q2_beta_imag >= 0 else fixed_q2_beta_imag + 2**16)

    # Pack into a 1D array of 16-bit unsigned integers
    unsigned_fixed_point_array = np.array([unsigned_q1_alpha_real, unsigned_q1_alpha_imag,
                                           unsigned_q1_beta_real, unsigned_q1_beta_imag,
                                           unsigned_q2_alpha_real, unsigned_q2_alpha_imag,
                                           unsigned_q2_beta_real, unsigned_q2_beta_imag], dtype=np.uint16)

    return unsigned_fixed_point_array


unsigned_fixed_point_data2 = convert_to_unsigned_fixed_point_16bit(alpha_real, alpha_imag, beta_real, beta_imag,
                                                                  alpha_real2, alpha_imag2, beta_real2, beta_imag2)

print(unsigned_fixed_point_data2)


[64869  1096 64397 64428 43150  4045  6765 22592]


In [104]:
def convert_to_unsigned_fixed_point_16bit_control(q1_alpha_real, q1_alpha_imag, q1_beta_real, q1_beta_imag,
                                                  control_alpha_real, control_alpha_imag, control_beta_real, control_beta_imag):
    # Convert both Qubit 1 and the control qubit to 16-bit signed fixed-point representation
    fixed_q1_alpha_real = np.int16(q1_alpha_real * (2**15))
    fixed_q1_alpha_imag = np.int16(q1_alpha_imag * (2**15))
    fixed_q1_beta_real = np.int16(q1_beta_real * (2**15))
    fixed_q1_beta_imag = np.int16(q1_beta_imag * (2**15))

    fixed_control_alpha_real = np.int16(control_alpha_real * (2**15))
    fixed_control_alpha_imag = np.int16(control_alpha_imag * (2**15))
    fixed_control_beta_real = np.int16(control_beta_real * (2**15))
    fixed_control_beta_imag = np.int16(control_beta_imag * (2**15))

    # Convert to unsigned by adding 2^16 to negative values
    unsigned_q1_alpha_real = np.uint16(fixed_q1_alpha_real if fixed_q1_alpha_real >= 0 else fixed_q1_alpha_real + 2**16)
    unsigned_q1_alpha_imag = np.uint16(fixed_q1_alpha_imag if fixed_q1_alpha_imag >= 0 else fixed_q1_alpha_imag + 2**16)
    unsigned_q1_beta_real = np.uint16(fixed_q1_beta_real if fixed_q1_beta_real >= 0 else fixed_q1_beta_real + 2**16)
    unsigned_q1_beta_imag = np.uint16(fixed_q1_beta_imag if fixed_q1_beta_imag >= 0 else fixed_q1_beta_imag + 2**16)

    unsigned_control_alpha_real = np.uint16(fixed_control_alpha_real if fixed_control_alpha_real >= 0 else fixed_control_alpha_real + 2**16)
    unsigned_control_alpha_imag = np.uint16(fixed_control_alpha_imag if fixed_control_alpha_imag >= 0 else fixed_control_alpha_imag + 2**16)
    unsigned_control_beta_real = np.uint16(fixed_control_beta_real if fixed_control_beta_real >= 0 else fixed_control_beta_real + 2**16)
    unsigned_control_beta_imag = np.uint16(fixed_control_beta_imag if fixed_control_beta_imag >= 0 else fixed_control_beta_imag + 2**16)

    # Pack into a 1D array of 16-bit unsigned integers (for Qubit 1 and control qubit)
    unsigned_control_fixed_point_array = np.array([ unsigned_control_alpha_real, unsigned_control_alpha_imag,
                                                   unsigned_control_beta_real, unsigned_control_beta_imag, unsigned_q1_alpha_real, unsigned_q1_alpha_imag,
                                                   unsigned_q1_beta_real, unsigned_q1_beta_imag], dtype=np.uint16)

    return unsigned_control_fixed_point_array

# Generate the control qubit (|1> state)
control_alpha_real, control_alpha_imag, control_beta_real, control_beta_imag = generate_one_qubit()

# Convert to unsigned fixed-point representation for Qubit 1 and control qubit
unsigned_control_fixed_point_data = convert_to_unsigned_fixed_point_16bit_control(alpha_real, alpha_imag, beta_real, beta_imag,
                                                                                 control_alpha_real, control_alpha_imag, control_beta_real, control_beta_imag)

print("Qubit 1 and Control Qubit (16-bit unsigned fixed-point representation):")
print(unsigned_control_fixed_point_data)

Qubit 1 and Control Qubit (16-bit unsigned fixed-point representation):
[    0     0 32768     0 64869  1096 64397 64428]


In [105]:
twoQubitEnable = 0
controlQubitEnable = 0
isHadamard = 0
def load_gate_bitstream():
    global twoQubitEnable
    global controlQubitEnable
    global isHadamard
    print("Select the gate you want to emulate:")
    print("- X\n- Y\n- Z\n- HADAMARD\n- S\n- T\n- SWAP")

    user_choice = input("Enter your gate choice: ").strip().upper()

    # Load the appropriate bitstream based on user input
    if user_choice == "X":
        ol = Overlay("XGate.bit")
    elif user_choice == "Y":
        ol = Overlay("YGate.bit")
    elif user_choice == "Z":
        ol = Overlay("ZGate.bit")
    elif user_choice == "HADAMARD":
        ol = Overlay("HadamardGate.bit")
        isHadamard = 1
    elif user_choice == "S":
        ol = Overlay("SGate.bit")
        twoQubitEnable = 1
        controlQubitEnable = 1
    elif user_choice == "T":
        ol = Overlay("TGate.bit")
        twoQubitEnable = 1
        controlQubitEnable = 1
    elif user_choice == "SWAP":
        ol = Overlay("SWAPGate.bit")
        twoQubitEnable = 1
    else:
        print("Invalid choice. Please select a valid gate.")
        return None

    # Display information about the loaded Overlay
    print(f"Successfully loaded the {user_choice} gate bitstream!")
    return ol

# Use the returned Overlay object
ol = load_gate_bitstream()
ol?
print(twoQubitEnable)
print(controlQubitEnable)
print(isHadamard)


Select the gate you want to emulate:
- X
- Y
- Z
- HADAMARD
- S
- T
- SWAP


Enter your gate choice:  hadamard


Successfully loaded the HADAMARD gate bitstream!
0
0
1


Type:            Overlay
String form:     <pynq.overlay.Overlay object at 0xffff5ab4c4f0>
File:            /usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/overlay.py
Docstring:      
Default documentation for overlay HadamardGate.bit. The following
attributes are available on this overlay:

IP Blocks
----------
axi_dma              : pynq.lib.dma.DMA
zynq_ultra_ps_e_0    : pynq.overlay.DefaultIP

Hierarchies
-----------
None

Interrupts
----------
None

GPIO Outputs
------------
None

Memories
------------
PSDDR                : Memory
Class docstring:
This class keeps track of a single bitstream's state and contents.

The overlay class holds the state of the bitstream and enables run-time
protection of bindings.

Our definition of overlay is: "post-bitstream configurable design".
Hence, this class must expose configurability through content discovery
and runtime protection.

The overlay class exposes the IP and hierarchies as attributes in the
overlay. If no other drivers

In [106]:
dma = ol.axi_dma

In [107]:
if twoQubitEnable == 0:
    # Allocate input and output buffers
    input_buffer = allocate(shape=(unsigned_fixed_point_data.size,), dtype=np.uint32)
    output_buffer = allocate(shape=(unsigned_fixed_point_data.size,), dtype=np.uint32)
elif controlQubitEnable == 1:
    # Allocate input and output buffers
    input_buffer = allocate(shape=(unsigned_control_fixed_point_data.size,), dtype=np.uint16)
    output_buffer = allocate(shape=(unsigned_control_fixed_point_data.size,), dtype=np.uint16)
else:
    # Allocate input and output buffers
    input_buffer = allocate(shape=(unsigned_fixed_point_data2.size,), dtype=np.uint16)
    output_buffer = allocate(shape=(unsigned_fixed_point_data2.size,), dtype=np.uint16)


In [108]:
if twoQubitEnable == 0:
    # Copying the data into `input_buffer`
    np.copyto(input_buffer, unsigned_fixed_point_data)
elif controlQubitEnable == 1:
     # Copying the data into `input_buffer`
    np.copyto(input_buffer, unsigned_control_fixed_point_data)
else:
    # Copying the data into `input_buffer`
    np.copyto(input_buffer, unsigned_fixed_point_data2)

In [109]:
start = time.time()

# Transfer Data
dma.recvchannel.transfer(output_buffer)
dma.sendchannel.transfer(input_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()

elapsed_time = time.time() - start
print("Total Time to calculate Gate = ", elapsed_time, " seconds" )

Total Time to calculate Gate =  0.0022215843200683594  seconds


In [110]:
output_buffer

PynqBuffer([3944696035, 1402333227, 2955569072, 4285945982], dtype=uint32)

In [111]:
def convert_from_unsigned_fixed_point(output_buffer):
    # Scaling factor for fixed-point conversion
    scaling_factor = 1 / (2**31)
    
    # Convert unsigned 32-bit values back to signed 32-bit values
    signed_alpha_real = np.int32(output_buffer[0] - 2**32 if output_buffer[0] >= 2**31 else output_buffer[0])
    signed_alpha_imag = np.int32(output_buffer[1] - 2**32 if output_buffer[1] >= 2**31 else output_buffer[1])
    signed_beta_real = np.int32(output_buffer[2] - 2**32 if output_buffer[2] >= 2**31 else output_buffer[2])
    signed_beta_imag = np.int32(output_buffer[3] - 2**32 if output_buffer[3] >= 2**31 else output_buffer[3])

    # Convert to floating-point by applying the scaling factor
    alpha_real = signed_alpha_real * scaling_factor
    alpha_imag = signed_alpha_imag * scaling_factor
    beta_real = signed_beta_real * scaling_factor
    beta_imag = signed_beta_imag * scaling_factor

    return alpha_real, alpha_imag, beta_real, beta_imag


In [112]:
def convert_from_unsigned_fixed_point_16bit(output_buffer):
    # Scaling factor for fixed-point conversion (since using 16 bits)
    scaling_factor = 1 / (2**15)

    def to_signed(value):
        """Convert unsigned to signed 16-bit value."""
        # If the value is 32768 (middle of the range), treat it as positive, not negative
        if value == 32768:
            return 32768
        elif value >= 2**15:
            return value - 2**16  # Convert to signed value
        return value
    
    # Convert the unsigned values to signed values
    signed_q1_alpha_real = to_signed(output_buffer[0])
    signed_q1_alpha_imag = to_signed(output_buffer[1])
    signed_q1_beta_real = to_signed(output_buffer[2])
    signed_q1_beta_imag = to_signed(output_buffer[3])

    signed_q2_alpha_real = to_signed(output_buffer[4])
    signed_q2_alpha_imag = to_signed(output_buffer[5])
    signed_q2_beta_real = to_signed(output_buffer[6])
    signed_q2_beta_imag = to_signed(output_buffer[7])

    # Convert to floating-point by applying the scaling factor
    q1_alpha_real = signed_q1_alpha_real * scaling_factor
    q1_alpha_imag = signed_q1_alpha_imag * scaling_factor
    q1_beta_real = signed_q1_beta_real * scaling_factor
    q1_beta_imag = signed_q1_beta_imag * scaling_factor

    q2_alpha_real = signed_q2_alpha_real * scaling_factor
    q2_alpha_imag = signed_q2_alpha_imag * scaling_factor
    q2_beta_real = signed_q2_beta_real * scaling_factor
    q2_beta_imag = signed_q2_beta_imag * scaling_factor

    
    return q1_alpha_real, q1_alpha_imag, q1_beta_real, q1_beta_imag, q2_alpha_real, q2_alpha_imag, q2_beta_real, q2_beta_imag


In [113]:
def convert_and_reverse_scaling(output_buffer, scale_factor=16):
    scaling_factor_fixed_point = 1 / (2**31)  # For converting back from signed fixed-point 32-bit
    reverse_scaling = 1 / scale_factor        # Reverse the scaling applied in Model Composer

    # Convert from unsigned back to signed 32-bit fixed-point
    signed_alpha_real = np.int32(output_buffer[0] - 2**32 if output_buffer[0] >= 2**31 else output_buffer[0])
    signed_alpha_imag = np.int32(output_buffer[1] - 2**32 if output_buffer[1] >= 2**31 else output_buffer[1])
    signed_beta_real = np.int32(output_buffer[2] - 2**32 if output_buffer[2] >= 2**31 else output_buffer[2])
    signed_beta_imag = np.int32(output_buffer[3] - 2**32 if output_buffer[3] >= 2**31 else output_buffer[3])

    # Apply both scaling corrections
    alpha_real = signed_alpha_real * scaling_factor_fixed_point * reverse_scaling
    alpha_imag = signed_alpha_imag * scaling_factor_fixed_point * reverse_scaling
    beta_real = signed_beta_real * scaling_factor_fixed_point * reverse_scaling
    beta_imag = signed_beta_imag * scaling_factor_fixed_point * reverse_scaling

    return alpha_real, alpha_imag, beta_real, beta_imag


In [114]:
if twoQubitEnable == 0:
    if isHadamard ==1: 
        # If Hadamard Gate selected- scaling must be reversed via appropiate function
        # Convert the random qubit Gate values from PL
        alpha_real, alpha_imag, beta_real, beta_imag = convert_and_reverse_scaling(output_buffer, scale_factor=16)

        print("alpha_real:", alpha_real)
        print("alpha_imag:", alpha_imag)
        print("beta_real:", beta_real)
        print("beta_imag:", beta_imag)
        
        # Convert the random qubit Gate values from PL
        alpha_real, alpha_imag, beta_real, beta_imag = convert_from_unsigned_fixed_point(output_buffer)

        print("alpha_real:", alpha_real)
        print("alpha_imag:", alpha_imag)
        print("beta_real:", beta_real)
        print("beta_imag:", beta_imag)

    else:
        # Convert the random qubit Gate values from PL
        alpha_real, alpha_imag, beta_real, beta_imag = convert_from_unsigned_fixed_point(output_buffer)

        print("alpha_real:", alpha_real)
        print("alpha_imag:", alpha_imag)
        print("beta_real:", beta_real)
        print("beta_imag:", beta_imag)
elif controlQubitEnable == 1:
    # Convert the 8x 16-bit values back to floating-point numbers
    control_alpha_real, control_alpha_imag, control_beta_real, control_beta_imag, q1_alpha_real, q1_alpha_imag, q1_beta_real, q1_beta_imag = convert_from_unsigned_fixed_point_16bit(output_buffer)

    print("Control Qubit - alpha_real:", control_alpha_real)
    print("Control Qubit - alpha_imag:", control_alpha_imag)
    print("Control Qubit - beta_real:", control_beta_real)
    print("Control Qubit - beta_imag:", control_beta_imag)
    
    # Print the results
    print("Qubit 1 - alpha_real:", q1_alpha_real)
    print("Qubit 1 - alpha_imag:", q1_alpha_imag)
    print("Qubit 1 - beta_real:", q1_beta_real)
    print("Qubit 1 - beta_imag:", q1_beta_imag)

else:
   # Convert the 8x 16-bit values back to floating-point numbers
    q1_alpha_real, q1_alpha_imag, q1_beta_real, q1_beta_imag, q2_alpha_real, q2_alpha_imag, q2_beta_real, q2_beta_imag = convert_from_unsigned_fixed_point_16bit(output_buffer)

    # Print the results
    print("Qubit 1 - alpha_real:", q1_alpha_real)
    print("Qubit 1 - alpha_imag:", q1_alpha_imag)
    print("Qubit 1 - beta_real:", q1_beta_real)
    print("Qubit 1 - beta_imag:", q1_beta_imag)

    print("Qubit 2 - alpha_real:", q2_alpha_real)
    print("Qubit 2 - alpha_imag:", q2_alpha_imag)
    print("Qubit 2 - beta_real:", q2_beta_real)
    print("Qubit 2 - beta_imag:", q2_beta_imag)

alpha_real: -0.01019423539401032
alpha_imag: 0.04081326848245226
beta_real: -0.03898161882534623
beta_imag: -0.00026255479315295815
alpha_real: -0.16310776630416512
alpha_imag: 0.6530122957192361
beta_real: -0.6237059012055397
beta_imag: -0.0042008766904473305
